Processing of Shop the Look data into Complete the Look dataset following [Kang et al (2019)](https://arxiv.org/abs/1812.01748)

In [89]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage import io
from PIL import Image
import os
import random
from tqdm import tqdm
import functools

DATA_PATH = './data/'

In [90]:
def convert_to_url(signature):
    prefix = 'http://i.pinimg.com/400x/%s/%s/%s/%s.jpg'
    return prefix % (signature[0:2], signature[2:4], signature[4:6], signature)

def get_product_img(data, idx):
    product_img_url = convert_to_url(data['product'][idx])
    return io.imread(product_img_url)

def get_scene_img(data, idx):
    scene_img_url = convert_to_url(data['scene'][idx])
    return io.imread(scene_img_url)

def get_bbox(data, idx):
    return data['bbox'][idx]
    
def display_img_pair(data, idx):
    product_img = get_product_img(data, idx)
    scene_img = get_scene_img(data, idx)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(product_img)
    axes[0].set_title('Product Image')
    axes[1].imshow(scene_img)
    axes[1].set_title('Scene Image')

    # get coordinates for normalized bounding box
    box = data['bbox'][idx]
    coords = box * np.array([scene_img.shape[1], scene_img.shape[0], scene_img.shape[1], scene_img.shape[0]])
    # convert to upper left and lower right
    rect = Rectangle((coords[0], coords[1]), coords[2] - coords[0], coords[3] - coords[1], linewidth=1, edgecolor='r', facecolor='none')

    axes[1].add_patch(rect)

    plt.show()

def get_category_name(data, idx, categories):
    item = data['product'][idx]
    if item in categories:
        return categories[item].split('|')[-1]
    else:
        return 'Unknown'

In [91]:
fashion = pd.read_json(DATA_PATH + 'fashion.json', lines=True)
categories = json.load(open(DATA_PATH + 'fashion-cat.json', 'r'))


In [92]:
def crop_size(image, bbox):
    width, height = image.shape[1], image.shape[0]
    left = bbox[0] * width
    top = bbox[1] * height
    right = bbox[2] * width
    bottom = bbox[3] * height

    # Calculate the four regions
    regions = [
        (0, 0, width, top),  # Top
        (0, bottom, width, height),  # Bottom
        (0, 0, left, height),  # Left
        (right, 0, width, height),  # Right
    ]

    # Find the largest region
    largest_region = max(regions, key=lambda r: (r[2]-r[0]) * (r[3]-r[1]))

    # compute size of cropped image
    cropped_width = largest_region[2] - largest_region[0]
    cropped_height = largest_region[3] - largest_region[1]

    return cropped_width, cropped_height

In [93]:
def crop_img(image, bbox):
    image = Image.fromarray(image)
    width, height = image.size
    left = bbox[0] * width
    top = bbox[1] * height
    right = bbox[2] * width
    bottom = bbox[3] * height

    # Calculate the four regions
    regions = [
        (0, 0, width, top),  # Top
        (0, bottom, width, height),  # Bottom
        (0, 0, left, height),  # Left
        (right, 0, width, height),  # Right
    ]

    # Find the largest region
    largest_region = max(regions, key=lambda r: (r[2]-r[0]) * (r[3]-r[1]))

    # Crop the image to the largest region
    cropped_image = image.crop(largest_region)

    return cropped_image


In [94]:
scene_to_product = {}
for row in fashion.itertuples():
    # row has product, scene, bbox
    scene_id = row.scene
    product_id = row.product
    if scene_id not in scene_to_product:
        scene_to_product[scene_id] = []
    scene_to_product[scene_id].append(product_id)

category_to_product = {}
for row in fashion.itertuples():
    product_id = row.product
    category = categories[product_id].split('|')[-1]
    if category not in category_to_product:
        category_to_product[category] = []
    category_to_product[category].append(product_id)


In [ ]:
ctl_data = pd.DataFrame(columns=['scene_id', 'product_id', 'bbox', 'category', 'label'])
for i in tqdm(range(len(fashion))):
    row = fashion.iloc[i]
    scene_id = row['scene']
    product_id = row['product']
    bbox = row['bbox']
    category = categories[product_id].split('|')[-1]

    # add to dataframe
    new_row = pd.DataFrame({'scene_id': [scene_id], 'product_id': [product_id], 'bbox': [bbox], 'category': [category], 'label': [1]})
    ctl_data = pd.concat([ctl_data, new_row], ignore_index=True)


    # add 3 negative samples
    for j in range(3):
        # pick random product that is the same category
        rand_product = random.choice(category_to_product[category])
        # check if product is in same scene
        if rand_product in scene_to_product[scene_id]:
            continue
        else:
            negative_row = pd.DataFrame({'scene_id': [scene_id], 'product_id': [rand_product], 'bbox': [bbox], 'category': [category], 'label': [0]})
            ctl_data = pd.concat([ctl_data, negative_row], ignore_index=True)


# save to csv
ctl_data.to_csv(DATA_PATH + 'ctl/data.csv', index=False)